In [ ]:
import pandas as pd
import requests
import json
import io
from dotenv import dotenv_values, load_dotenv

# Load credentials from .env file
load_dotenv()  
USERNAME, PASSWORD = dotenv_values().values()

# Set base path for API calls
BASE_URL = 'https://www-genesis.destatis.de/genesisWS/rest/2020/'

In [ ]:
whoami = requests.get(BASE_URL + 'helloworld/whoami', params={
    'username': USERNAME,
#    'password': PASSWORD,
    'language': 'en'
})
whoami.text

In [ ]:
hello = requests.get(BASE_URL + 'helloworld/logincheck', params={
    'username': USERNAME,
    'password': PASSWORD,
    'language': 'en'
})
json.dumps(hello.json(), indent=2)


In [ ]:
search = requests.get(BASE_URL + 'find/find', params={
    'username': USERNAME,
    'password': PASSWORD,
    'language': 'de',
    'term': 'Inflation'
})
search_results = json.loads(search.text)
search_results['Status']

In [ ]:
pd.DataFrame.from_dict(search_results['Statistics'])


In [ ]:
pd.DataFrame.from_dict(search_results['Tables'])


In [ ]:
response = requests.get(BASE_URL + 'data/tablefile', params={
    'username': USERNAME,
    'password': PASSWORD,
    'name': '61111-0002',
    'area': 'all',
    'compress': 'true',
    'format': 'ffcsv',
    'language': 'de'
})

In [ ]:
import zipfile
zf = zipfile.ZipFile(io.BytesIO(response.content))
file = zf.open(zf.filelist[0].filename)

tabelle61111_0002neu = pd.read_csv(file, sep=";",
                                   decimal=",", na_values=["-", "x", ".", "/"]) \
    .sort_values(by=["time","value_variable_label"])


In [ ]:
last_row = tabelle61111_0002neu.loc[tabelle61111_0002neu['value_variable_label'] == "Verbraucherpreisindex"].sort_values(by=["time","1_variable_attribute_code"]).tail(1)

In [ ]:
year = last_row['time'].values[0]
month = last_row['1_variable_attribute_label'].values[0]
value = last_row['value'].values[0]
last_row


In [ ]:
print(year, month, value)